# HW1: KNN classification

In this homework assignment, we'll implement a KNN classifier using elementary Python functionality. We will then train and test the classifier on phonetic data and compare our implementation to the sklearn implementation `sklearn.neighbors.KNeighborsClassifier`.

Before you start, rename the notebook as `HW1_KNN_firstname.ipynb`. E.g. `HW1_KNN_miikka.ipynb`.

Our tasks include:

1. Reading training and test data in pickle format
1. Implementing distance functions for kNN
1. Majority baseline
1. Implementing the kNN classifier
1. Tuning the k parameter for kNN
1. Comparing your implementation to sklearn
1. Evaluation

Grading criteria for code:

* Readability of code
* Correctness of code and solution
* Appropriate code commenting
* Your code should be reasonably efficient. No need to optimize too much but it should be possible to run the entire notebook in 15 minutes.

Grading criteria for half-page report (in assignment 7. Evaluation):

* Clear explanation of what you did
* Clear explanation of your observations concerning the output of the classifier.

The **deadline for this assignment is Sunday October 16 at 23:59 PT**. Please upload your notebook `HW1_KNN_firstname.ipynb` on Canvas by the deadline. No need to include the datasets.

Let's start by importing some useful modules:

In [2]:
from collections import Counter

import pickle
import sklearn.metrics
import sklearn.preprocessing
import sklearn.neighbors
import sklearn.dummy

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


## 1. Reading data (15 points)

You should download the datafiles `phon.train.pkl` and `phon.test.pkl` from Canvas and upload them to Google Colab (if you complete this homework on Colab).

Start by reading the datasets into two variables `train_and_dev` (containing the training set) and `test` (containing the test set) using the `pickle` Python built-in module.

In [3]:
# your code here
with open("phon.train.pkl","rb") as file:
    train_and_dev = pickle.load(file)
with open("phon.test.pkl","rb") as file:
    test = pickle.load(file)

Print the first training example and the length of the training and test set.

In [4]:
# your code here
print("first training example: ", train_and_dev[0])
print("length of training and dev: ", len(train_and_dev))
print("length of test: ", len(test))

first training example:  {'vowel': 'AH', 'word': 'COVER', 'F1': 608.5, 'F2': 1322.2}
length of training and dev:  2804
length of test:  700


Since there is no separate development file, we will need to start by separating the train set into a training and development split. Store the 300 first examples in `train_and_dev` into a list called `dev` and the remaining examples in a list called `train`.

You can check that the combined length of `train` and `dev` is the same as the length of the `train_and_dev` list. 

In [5]:
# your code here
dev = train_and_dev[:300]
print("length of dev: ", len(dev))
train = train_and_dev[300:]
print("length of train: ", len(train))
print("length of separated train and dev: ", len(dev)+len(train))

length of dev:  300
length of train:  2504
length of separated train and dev:  2804


Here are a few [assertions](https://www.w3schools.com/python/ref_keyword_assert.asp) to check that your code is working properly

In [6]:
# Assertions to check your code
assert(len(train) == 2504)
assert(len(dev) == 300)
assert(len(test) == 700)

The examples in `train`, `dev` and `test` are Python dictionaries:
    
```
{'vowel': 'ER', 'word': 'FLOWER', 'F1': 712.0, 'F2': 1715.5}
```

Each example contains a `"vowel"` field which tells you the true vowel corresponding to this example. The examples also contain two format frequencies `"F1"` and `"F1"`. Our task is to build a classifier which can classify examples by vowel based on their formant frequencies.

We will now write some data preprocessing code. The aim is to convert examples in dictionary format to an input tuple and output label:

```
input: (712.0, 1715.5), output: 0
```

The first value in the tuple represents the F1 formant and the second one represents the F2 formant. The output is a numerical code representing the vowel (e.g. "ER").

You should use [`sklearn.preprocessing.LabelEncoder()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) to convert your vowel labels to numerical labels. Start by initializing the label encoder. If you need help, check the KNN practical work from lecture 5. You can then call the `fit()` function of the label encoder on a list containing all the vowel labels (e.g. `"ER"`) to train the label encoder. Store the labels for the train set in a list `train_y`, for the development set in `dev_y` and for the test set in `test_y`.

You should store the input examples (e.g. `(712.0, 1715.5)`) in the lists `train_X`, `dev_X`, `test_X`.

Miikka's first train example looks like this (you class number might be different):
```
>>> print(train_X[0])
(712.0, 1715.5)

>>> print(train_y[0])
7
```

In [11]:
# your code here
label_encoder = sklearn.preprocessing.LabelEncoder()
train_label = [item["vowel"] for item in train]
dev_label = [item["vowel"] for item in dev]
test_label = [item["vowel"] for item in test]
label_encoder.fit(train_label+dev_label+test_label)

def get_input_output(data, label_encoder):
    "get X from datasets and y from label_encoder"
    X=list()
    y=list()
    for item in data:
        f1 = float(item["F1"])
        f2 = float(item["F2"])
        X.append((f1, f2))
        y.append(label_encoder.transform([item["vowel"]])[0])
    return X, y
train_X, train_y = get_input_output(train, label_encoder)
dev_X, dev_y = get_input_output(dev, label_encoder)
test_X, test_y = get_input_output(test, label_encoder)

A few assertions to check that your code works properly:

In [12]:
# Assertions to check your code

import numbers
assert len(train_X[0]) == 2
assert isinstance(train_X[0][0], numbers.Number)
assert isinstance(train_X[0][1], numbers.Number)
assert isinstance(train_y[0], numbers.Integral)

How many classes does the dataset contain? You can check this by printing the length of the list `label_encoder._classes`

In [13]:
# your code here
print("type of vowels: ", label_encoder.classes_)
print("number of vowels: ", len(label_encoder.classes_))

type of vowels:  ['AA' 'AE' 'AH' 'AO' 'AW' 'AY' 'EH' 'ER' 'EY' 'IH' 'IY' 'OW' 'OY' 'UH'
 'UW']
number of vowels:  15


## 2. Distance functions (5 points)

You should now implement two distance functions `euclidean_dist()` and `manhattan_dist()`. Both take two lists of numbers `x1` and `x2` as input and return their Euclidean and Mahattan distance, respectively. E.g.

```
euclidean_dist((1,2,3,4), (2,3,4,5)) == 2
```

You should not rely on library implementations of these functions. Implement them yourself.

In [14]:
# your code here
def euclidean_dist(coord1, coord2):
    "calculate the euclidean distance between two coordinates"
    sqr_dist = 0
    for x, y in zip(coord1, coord2):
        sqr_dist += (x-y)**2
    return sqr_dist ** 0.5

def manhattan_dist(coord1, coord2):
    "calculate the manhattan distance between two coordinates"
    man_dist = 0
    for x, y in zip(coord1, coord2):
        man_dist += abs(x-y)
    return man_dist

A few assertions to check that your code works properly.

In [15]:
# Assertions to check your code
assert euclidean_dist((1,1,1), (1,1,1)) == 0
assert euclidean_dist((1,2,3,4), (2,3,4,5)) == 2
assert euclidean_dist((2,3,4,5), (1,2,3,4)) == 2
assert manhattan_dist((1,1,1), (1,1,1)) == 0
assert manhattan_dist((1,2,3,4), (2,3,4,5)) == 4
assert manhattan_dist((2,3,4,5), (1,2,3,4)) == 4

## 3. Majority baseline (5 points)

Implement a majority baseline and evaluate it on the development data. You are allowed to use [`sklearn.dummy.DummyClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html).

Your accuracy should be around `13%`.

In [16]:
# your code here
baseline = sklearn.dummy.DummyClassifier(strategy="most_frequent")
baseline.fit(train_X, train_y)

baseline.score(dev_X, dev_y)

0.12666666666666668

## 4. KNN classifier (20 points)

We'll now implement a function `knn_classify_ex` which takes the following parameters:

* `x` an example like `(505.1, 1201.5)`
* `train_X` and `train_y` are the training examples (inputs and classes)
* `k` represents the k parameter for the kNN algorithm
* dist_func is either your Euclidean distance function `euclidean_dist` or Manhattan distance function `manhattan_dist`.

The function `knn_classify_ex` finds the `k` datapoints in `train_X` which are closest to `x`. It then finds their classes and returns the majority class. E.g. if `k == 3` and among the 3 closest points to x there are 2 examples with class `0` and one with class `1`, then you should return 0. Importantly, check that the function returns a single class number.

Especially for smaller values of `k`, ties can easily happen and there might not exist a clear majority class. In this case, you may return an arbitrary class chosen among the most frequent classes. 

**Hint:** There are several ways to implement this function but it can be a good idea to first form a list of `(distance, class)` pairs and then sort this list into descending numerical order:

```
>>> l = [(1,0), (1.1,1), (0.1, 2), (0.5, 2)]
>>> l.sort()
print(l)
```

**Hint:** [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) from the Python `collections` module can be useful for majority voting.

In [17]:
# your code here
def knn_classify_ex(x, train_X, train_y, k, dist_func):
    "build knn classifier for one example"
    dist_class = list()
    for fmts, vowel in zip(train_X, train_y):
        if dist_func == "euclidean":
            dist = euclidean_dist(x, fmts)
        elif dist_func == "manhattan":
            dist = manhattan_dist(x, fmts)
        dist_class.append((dist, vowel)) # get (distance, class) list
    dist_class.sort() # sort the list by distance
    nearest_classes = [ex[1] for ex in dist_class[:k]] # get the classes for the k nearest items
    pred_class = Counter(nearest_classes).most_common(1)[0][0] # get the majority class
    return pred_class
    

You should now implement a function `knn_classify` which applies `knn_classify_ex` to every example in a dataset. It takes the following parameters:

* `data` a dataset (in our case this will be either the development or test data).
* `train_X` and `train_y` see above
* `k` see above
* `dist_func` see above 

The function returns a list of classes, one for each example in `data`.

In [18]:
# your code here
def knn_classify(data, train_X, train_y, k, dist_func):
    "build knn classifier for a dataset"
    pred_y = list()
    for ex in data:
        y = knn_classify_ex(ex, train_X, train_y, k, dist_func)
        pred_y.append(y)
    return pred_y

Run `knn_classify` on the development set `dev_X` using `k == 1` and `euclidean_dist`. Store the output in the list `sys_dev_y`. Then use [`sklearn.metrics.accuracy_score`]() to find the accuracy of the 1NN classifier. 

Your accuracy should exceed 20%.

In [19]:
# your code here
sys_dev_y = knn_classify(dev_X, train_X, train_y, 1, "euclidean")
sklearn.metrics.accuracy_score(dev_y, sys_dev_y)


0.26666666666666666

## 5. Tuning the k parameter (15 points)

Train kNN classifiers for k in the range 1-50 and using different distance functions (Euclidean or Manhattan). For each k, check the accuracy of the classifier on the development set. Identify the best k and distance function based on development accuracy. 

Your best accuracy should be substantially higher than for the 1NN classifier. You should be able to get above 40%. 

Since we are running 100 experiments, it is important that your implementation of `knn_classify_ex` is reasonably fast. If it is, running all the experiments can take around five minutes but should not take much longer.

In [20]:
# your code here
accuracy_k_dist_func = list() 
for k in range(1, 51):
    for dist_func in ["euclidean", "manhattan"]:
        pred_dev_y = knn_classify(dev_X, train_X, train_y, k, dist_func)
        accuracy = sklearn.metrics.accuracy_score(dev_y, pred_dev_y)
        accuracy_k_dist_func.append((accuracy, k, dist_func))
accuracy_k_dist_func.sort(reverse=True)
print(accuracy_k_dist_func[:10])

[(0.42333333333333334, 20, 'euclidean'), (0.42, 44, 'euclidean'), (0.42, 39, 'euclidean'), (0.4166666666666667, 42, 'euclidean'), (0.4166666666666667, 40, 'euclidean'), (0.4166666666666667, 38, 'euclidean'), (0.41333333333333333, 45, 'euclidean'), (0.41333333333333333, 43, 'euclidean'), (0.41333333333333333, 41, 'euclidean'), (0.41333333333333333, 37, 'euclidean')]


Run the same experiment but this time using F-score instead of accuracy. You can use [`sklearn.metrics.f1_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html). Again, identify the best k and distance function which maximize the F-score. Are these the same as for accuracy?

In [21]:
# your code here
f1_k_dist_func = list() 
for k in range(1, 51):
    for dist_func in ["euclidean", "manhattan"]:
        pred_dev_y = knn_classify(dev_X, train_X, train_y, k, dist_func)
        f1 = sklearn.metrics.f1_score(dev_y, pred_dev_y, average="macro")
        f1_k_dist_func.append((accuracy, k, dist_func))
f1_k_dist_func.sort(reverse=True)
print(f1_k_dist_func[:10])

# f1 score is lower than the accuracy score for the same k and distance function

[(0.39, 50, 'manhattan'), (0.39, 50, 'euclidean'), (0.39, 49, 'manhattan'), (0.39, 49, 'euclidean'), (0.39, 48, 'manhattan'), (0.39, 48, 'euclidean'), (0.39, 47, 'manhattan'), (0.39, 47, 'euclidean'), (0.39, 46, 'manhattan'), (0.39, 46, 'euclidean')]


## 6. Comparison to sklearn (5 points)

Train an [`sklearn.neighbors.KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) on your `train_X` and `train_y` data using your best k and best distance metric (identified using accuracy, not F-score).

If you are using Euclidean distance, you can initialize the KNN model with the parameter `metric="euclidean"`. If you are using Manhattan distance, initialize using `metric="cityblock"`.

Check the classification accuracy on the development set. It should be very close your performance. If the accuracy is off by more than 5%-points, something might be wrong.

In [23]:
# your code here
classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=20,weights="uniform", metric="euclidean")
classifier.fit(train_X, train_y)

sys_dev_y = classifier.predict(dev_X)

skl_knn_accuracy = sklearn.metrics.accuracy_score(dev_y, sys_dev_y)
print(skl_knn_accuracy)
print("difference bwteen self-built knn and knn in sklearn is:", round(skl_knn_accuracy-accuracy_k_dist_func[0][0],2))

0.43333333333333335
difference bwteen self-built knn and knn in sklearn is: 0.01


## 7. Evaluation (10 points)

Print the classification report both for your development data and your test data. 

In [24]:
# your code here
# report for development data
print("----------- Classification report for development data -----------")
print(sklearn.metrics.classification_report(dev_y, sys_dev_y))
# report for test data
sys_test_y = classifier.predict(test_X)
print("----------- Classification report for test data -----------")
print(sklearn.metrics.classification_report(test_y, sys_test_y, target_names = label_encoder.classes_))

----------- Classification report for development data -----------
              precision    recall  f1-score   support

           0       0.29      0.15      0.20        13
           1       0.50      0.57      0.53        14
           2       0.23      0.11      0.15        27
           3       0.50      0.43      0.46        21
           4       0.00      0.00      0.00         9
           5       0.47      0.86      0.61        22
           6       0.28      0.41      0.33        17
           7       0.39      0.53      0.45        30
           8       0.36      0.52      0.43        23
           9       0.41      0.39      0.40        33
          10       0.77      0.71      0.74        38
          11       0.33      0.37      0.35        19
          13       0.00      0.00      0.00         5
          14       0.37      0.24      0.29        29

    accuracy                           0.43       300
   macro avg       0.35      0.38      0.35       300
weighted avg 

C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Write a short report (half a page) of what you did in this assignment. Please also include an analysis of the classification reports. 

* Do the overall accuracy and f-score agree in the development and test data? 
* Are some of the classes particularly easy/hard to classify? Why do you think that might be?
* Are there any classes which behave differently in the development and test set? 
* If there are, why do you think they behave differently?

YOUR ANSWER HERE
## Assignment summary
### Overview and workflow
This assignment aimed to build a knn classifier from scratch to classify vowels based on the first and second formant values (F1 and F2). Performance of my knn classifier was then compared to the knn classifier from the sklearn package. 
In this assignment, I completed the following tasks.
1. Loaded train and test data in .pkl format and stored data in train (2804 examples) and test (700 examples).
2. Splited the train data in to train dataset (2504 examples) and development dataset (300 examples).
3. Got input (F1 and F2) and output (vowel in integer classes) values for train, dev, and test dataset.
4. Defined two distance calculation function, euclidean and manhattan.
5. Got the majority baseline by referring to the most frequent vowel class.
6. Built a knn classifier. This was done by calculating the distance (euclidean or manhattan) from one data point to all other points in the train dataset, and then assigned the majority class of the k nearest data points as the predicted class. This process was applied to all data points to the dev dataset.
7. Ran the knn classifier with different k and distance calculation methods to get the best performance. 
8. Run knn classifier from the sklearn package with the best k and distance calculation function and compared with my knn performance.
### Discussion
* The overall accuracy and f-score agree in the development and test dataset. The accuracy and macro and micro average f1-score in the development dataset is slightly higher than those in the test dataset. Possibly beacuse the test dataset contains more data points which might have induce more variability
* Based on the classification report for both development and test dataset, it seems the vowel IY has high precision score. It possibly because that IY is a tensed high front vowel that was produced with very small variation and is not overlapped with over vowels in the vowel space. The UH has harder to classify. I inspected the examples for UH and UW and I noticed that a lot of UH was mislabeled as UW (e.g. vowel in to should be UH but labeled as UW), which resulted in high variability in the UW formants.
* Classifier performance for the AE vowel varied a lot between development and test set. 
* I calculate the standard derivation of F1 and F2 of this vowel in the development and test set, and I noticed that std in F2 is larger in the test set (257) than the development set (103). This indicate more variablity in the test set, which might results in more AE being misclassified as another adjacent vowel by the knn classifier.

In [7]:

def get_formants(data, vowel):
    f1 = []
    f2 = []
    for ex in data:
        if ex["vowel"] == vowel:
            f1.append(ex["F1"])
            f2.append(ex["F2"])
    return f1, f2

train_f1, train_f2 = get_formants(train, "AE")
dev_f1, dev_f2 = get_formants(dev, "AE")
test_f1, test_f2 = get_formants(test, "AE")

In [8]:
import pandas as pd
f1 = pd.Series(train_f2)
f1.describe()

count     123.000000
mean     1794.457724
std       313.799404
min      1245.900000
25%      1584.550000
50%      1702.100000
75%      1984.400000
max      2959.000000
dtype: float64

In [9]:
f1 = pd.Series(dev_f2)
f1.describe()

count      14.000000
mean     1551.178571
std       103.143374
min      1374.500000
25%      1501.250000
50%      1523.550000
75%      1635.800000
max      1751.500000
dtype: float64

In [10]:
f1 = pd.Series(test_f2)
f1.describe()

count      38.000000
mean     1697.260526
std       257.298670
min      1315.800000
25%      1522.100000
50%      1595.100000
75%      1860.000000
max      2344.600000
dtype: float64